In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import r2_score

In [2]:
names = ['Trial 1','Trial 2','Trial 3','Trial 4','Trial 5','Trial 6','Trial 7','Trial 8','Trial 9','Trial 10']

dfR2mean3 = pd.read_excel('20240823 Predicting Optimum Temperature  - Simple Guessing.xlsx', index_col =0,
                          sheet_name =names)
dfModels = pd.read_excel('20240822 Predicting Optimum Temperature - y_val values.xlsx', index_col = 0, sheet_name =names)

In [3]:
list_merged_results = []

for name in names:
     dfModels[name].insert(loc=0, column='Simple Guessing', value=dfR2mean3[name]['Average'].values)
    

In [4]:
def analyze_results(summary):
    
    identity_less100 = summary[summary['Sequence Identity'] < 1] 
    identity_80 = summary[(summary['Sequence Identity'] < 0.9999) & (summary['Sequence Identity'] > 0.8)]
    identity_0 = summary[summary['Sequence Identity'] < 0.7999]
    identity_80_99 = summary[(summary['Sequence Identity'] <= 0.99) & (summary['Sequence Identity'] > 0.8)]
    identity_99 = summary[summary['Sequence Identity'] > 0.99] 



    identity_less100_w = summary[(summary['Sequence Identity'] < 1) & (summary['wild type or mutant'] == 'wild')] 
    identity_80_w = summary[(summary['Sequence Identity'] < 0.9999) & (summary['Sequence Identity'] > 0.8) &
                                                               (summary['wild type or mutant'] == 'wild')]
    identity_0_w = summary[(summary['Sequence Identity'] < 0.7999) & (summary['wild type or mutant'] == 'wild')]
    identity_80_99_w = summary[(summary['Sequence Identity'] <= 0.99) & (summary['Sequence Identity'] > 0.8) &
                                                                      (summary['wild type or mutant'] == 'wild')]
    identity_99_w = summary[(summary['Sequence Identity'] > 0.99) & (summary['wild type or mutant'] == 'wild')]  


    identity_less100_m = summary[(summary['Sequence Identity'] < 1) & (summary['wild type or mutant'] == 'mutant')] 
    identity_80_m = summary[(summary['Sequence Identity'] < 0.9999) & (summary['Sequence Identity'] > 0.8) &
                                                                    (summary['wild type or mutant'] == 'mutant')]
    identity_0_m = summary[(summary['Sequence Identity'] < 0.7999) & (summary['wild type or mutant'] == 'mutant')]
    identity_80_99_m = summary[(summary['Sequence Identity'] <= 0.99) & (summary['Sequence Identity'] > 0.8) &
                                                                      (summary['wild type or mutant'] == 'mutant')]
    identity_99_m = summary[(summary['Sequence Identity'] > 0.99) & (summary['wild type or mutant'] == 'mutant')]  




    matrix = [identity_less100, identity_80 ,identity_0 ,identity_80_99, identity_99 ,
              identity_less100_w, identity_80_w, identity_0_w, identity_80_99_w , identity_99_w,
              identity_less100_m,identity_80_m,identity_99_m]
    list_R2=[]

    for identity in matrix:
        for model in summary.columns[:-1]:
            r2=r2_score(identity['Temperature Optimum'], identity[model])
            list_R2.append(r2)



    x = np.reshape(list_R2, (len(matrix), len(summary.columns[:-1])))

    number_of_data = [len(a) for a in matrix]

    Results=pd.DataFrame(x, columns=summary.columns[:-1], index = ['<100%', '80-99.99%', '0-79.99%', '80-99%', '>99%',
                                                              '<100% wild type','80-99.99% wild type', '0-79.99% wild type', '80-99% wild type', '>99% wild type',
                                                              '<100% mutant type','80-99.99% mutant type', '>99% mutant type'])
    Results['Number of Data'] = number_of_data

    return Results

In [5]:
list_Results = []

for trial in dfModels:
    dfR = analyze_results(dfModels[trial])
    list_Results.append(dfR)

C:\Users\memre\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\memre\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\memre\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\memre\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\memre\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn

In [6]:
df_average = pd.concat([list_Results[0],list_Results[1],list_Results[2],list_Results[3], list_Results[4],
                       list_Results[5],list_Results[6],list_Results[7],list_Results[8], list_Results[9]])

average = df_average.groupby(level=0, sort=False).mean()
std = df_average.groupby(level=0, sort=False).std()
list_Results.append(average)
list_Results.append(std)

In [7]:
names = ['Trial 1','Trial 2','Trial 3','Trial 4','Trial 5','Trial 6','Trial 7','Trial 8','Trial 9','Trial 10', "Average", "STD", 'Number of Data Points']
writer=pd.ExcelWriter(r"C:\Users\memre\Desktop\Research\Predicting Enzyme Properties Based on Various Organisms\Code\Temperature Profile Prediction\20240823 Predicting Optimum Temperature betaGlucosidase - Comparing with Simple Guessing.xlsx")
_ = [A.to_excel(writer,sheet_name="{0}".format(names[i])) for i, A in enumerate(list_Results)]
writer.close()